In [5]:
import cv2
import dlib
from imutils.face_utils import FaceAligner
import matplotlib.pyplot as plt 

In [6]:
"""

1. pose_predictor is used to get the 68 facial landmarks detection.
2. face_encoder is used to get the (128,1) dimension encoding of the image which is passed to it.
   It is a pretrained ResNet network model with 29 conv layers. The model is trained on a dataset of about 3 million faces.
3. detector is an object of dlib.get_frontal_face_detector() which is used to get the front face from the face image.
4. modelFile, configFile are the files for the dnn based cv2 face detection model.
   It is a quantized tensorflow model which is based on the Single Shot-Multibox Detector (SSD) and uses ResNet-10 architecture as its backbone. It was introduced post OpenCV 3.3 in its deep neural network module.
5. net is the object of the cv2.dnn module that is initialized with the model and config file.

"""
print('', end='')

In [7]:
pose_predictor=dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

fa = FaceAligner(pose_predictor)

face_encoder=dlib.face_recognition_model_v1('dlib_face_recognition_resnet_model_v1.dat')

detector = dlib.get_frontal_face_detector()

modelFile = 'opencv_face_detector_uint8.pb'

configFile = 'opencv_face_detector.pbtxt'

net = cv2.dnn.readNetFromTensorflow(modelFile, configFile)

In [10]:
cap = cv2.VideoCapture(0)
while True:
    _, img = cap.read()
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    frameHeight = img.shape[0]
    frameWidth = img.shape[1]
    
    blob = cv2.dnn.blobFromImage(img, 1.0, (300, 300), [104, 117,   123], False, False)
    net.setInput(blob)
    detections = net.forward()
    
    frame = img
    
    for i in range(detections.shape[2]):
        
        confidence = detections[0, 0, i, 2]
        
        if confidence > 0.75:
           
            x1 = int(detections[0, 0, i, 3] * frameWidth)
            y1 = int(detections[0, 0, i, 4] * frameHeight)
            x2 = int(detections[0, 0, i, 5] * frameWidth)
            y2 = int(detections[0, 0, i, 6] * frameHeight)
            
            faceAligned =fa.align(img, gray,dlib.rectangle(x1,y1,x2,y2))
            landmark = pose_predictor(faceAligned,dlib.rectangle(0,0,faceAligned.shape[0],faceAligned.shape[1]))
            
            for i in range(0, 68):
                x = landmark.part(i).x
                y = landmark.part(i).y
                cv2.circle(faceAligned, (x, y), 1, (255, 255, 255), -1)
            face_descriptor = face_encoder.compute_face_descriptor(faceAligned, landmark, num_jitters=2)

            
            frame = faceAligned
            
            
    cv2.imshow("Frame", frame)
    key =  cv2.waitKey(1)
    if key == 27:
        break
cap.release()
cv2.destroyAllWindows()
